DATA COLLECTION

In [20]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import keras
import numpy as np

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range = 0.2, zoom_range =[.99, 1.01], brightness_range = [0.8, 1.2],data_format="channels_last",fill_mode="constant",horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

IMAGE PREPROCESSING

In [22]:
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input 
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
from tensorflow.keras.models import load_model 
from tensorflow.keras.preprocessing import image
import numpy as np

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
train_datagen = ImageDataGenerator(rescale = 1./255, shear_range= 0.2, zoom_range = [.99, 1.01], brightness_range= [0.8, 1.2], data_format= "channels_last", fill_mode="constant", horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [33]:
training_set = train_datagen.flow_from_directory('../dataset', target_size = (180, 180), batch_size = 64, class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('../dataset',target_size = (180, 180), batch_size = 64, class_mode = 'categorical')



Found 1530 images belonging to 6 classes.
Found 1530 images belonging to 6 classes.


MODEL BUILDING

In [25]:
Image_SIZE = [180, 180]
VGG19 = VGG19(input_shape = Image_SIZE +[3],weights = 'imagenet',include_top = False )

In [26]:
for layer in VGG19.layers:
  layer.trainable = False
x = Flatten()(VGG19.output)
prediction = Dense(6, activation = 'softmax')(x)
model = Model(inputs = VGG19.input, outputs = prediction)

In [27]:
model.compile(optimizer="sgd",
              loss="categorical_crossentropy",
              metrics=[keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.SpecificityAtSensitivity(0.5), keras.metrics.SensitivityAtSpecificity(0.5), 'accuracy'])

In [28]:
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

Epoch 1/5


2023-05-20 01:01:33.009350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - ETA: 0s - loss: 9.3950 - precision: 0.2273 - recall: 0.2111 - specificity_at_sensitivity: 0.0000e+00 - sensitivity_at_specificity: 0.4680 - accuracy: 0.2222 

2023-05-20 01:05:59.952824: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - 528s 22s/step - loss: 9.3950 - precision: 0.2273 - recall: 0.2111 - specificity_at_sensitivity: 0.0000e+00 - sensitivity_at_specificity: 0.4680 - accuracy: 0.2222 - val_loss: 10.1884 - val_precision: 0.1961 - val_recall: 0.1961 - val_specificity_at_sensitivity: 0.0000e+00 - val_sensitivity_at_specificity: 0.2856 - val_accuracy: 0.1961
Epoch 2/5
24/24 [==============================] - 542s 23s/step - loss: 10.0466 - precision: 0.2413 - recall: 0.2392 - specificity_at_sensitivity: 0.0000e+00 - sensitivity_at_specificity: 0.4582 - accuracy: 0.2405 - val_loss: 8.3760 - val_precision: 0.4271 - val_recall: 0.4229 - val_specificity_at_sensitivity: 0.7736 - val_sensitivity_at_specificity: 0.5529 - val_accuracy: 0.4248
Epoch 3/5
24/24 [==============================] - 595s 25s/step - loss: 5.0018 - precision: 0.3902 - recall: 0.3634 - specificity_at_sensitivity: 0.8068 - sensitivity_at_specificity: 0.6601 - accuracy: 0.3791 - val_loss: 4.3609 - val_pre

In [30]:
result= model.evaluate(test_set,
                               steps = 11,
                               verbose = 2,
                               use_multiprocessing = True,
                               workers = 2)
loss = result[0]
accuracy = result[1]
print('Model performance on test images:\nAccuracy = {accuracy}\nLoss = {loss}')

2023-05-20 01:52:26.814284: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


11/11 - 122s - loss: 1.9423 - precision: 0.4836 - recall: 0.4602 - specificity_at_sensitivity: 0.8886 - sensitivity_at_specificity: 0.9105 - accuracy: 0.4787 - 122s/epoch - 11s/step
Model performance on test images:
Accuracy = {accuracy}
Loss = {loss}


In [31]:
model.save('WCV.h5')

In [32]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
model = load_model('./WCV.h5')